## Common code and data loading

In [ ]:
#ONLY RUN THIS IF THE GPU SERVER IS BUSY
#USES THE SLOWASS CPU INSTEAD

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [1]:
#Run this to choose 2nd GPU

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [14]:
# Run this if the following error is thrown when training/testing models:

# Error : Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, 
# so try looking to see if a warning log message was printed above.

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
from keras.backend import manual_variable_initialization

import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config_tf()


def config_tf():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    set_session(sess)
    
config_tf()

In [3]:

import SimpleITK as sitk
import os, sys
#sys.path.insert(1, './Models/Resnet-3D')
from resnet3d4 import Resnet3DBuilder
import resnet3d4

import numpy as np
import random
import keras
import math
import time
import utils

import keras.backend as K
from keras.models import model_from_json #, load_model
from keras import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint

from scipy.spatial import distance
from scipy.ndimage.morphology import binary_fill_holes
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

import matplotlib.pyplot as plt

#these are imported or the transferring models are not deserialised properly
import conv4d
from keras.regularizers import l2

In [4]:
%load_ext autoreload
%autoreload 2

#if this is not here, the loaded notebook will not detect ensuing changes in imported numpy scripts 

In [5]:
#load training data from disk

pet_data, ct_data, centre_data, size_data, mask_data, ct_radiomics_data, pet_radiomics_data = {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, \
                                                        {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, {'train':{}, 'test':{}}, \
                                                        {'train':{}, 'test':{}}, {'train':{}, 'test':{}}

for dataset in ['HeadNeckCancer', 'BreastCancer']:
    pet_files = []
    ct_files = []
    centre_files = []
    size_files = []
    mask_files = []
    pet_radiomics_files = []
    ct_radiomics_files = []
    
    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/PET'):
        for name in files:
            file_path = os.path.join(root, name)
            pet_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/CT'):
        for name in files:
            file_path = os.path.join(root, name)
            ct_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/MaskCentres'):
        for name in files:
            file_path = os.path.join(root, name)
            centre_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/MaskSizes'):
        for name in files:
            file_path = os.path.join(root, name)
            size_files.append(file_path)

    for root, dirs, files in os.walk('/home/jzhe0882/numpydata/' + dataset + '/Mask'):
        for name in files:
            file_path = os.path.join(root, name)
            mask_files.append(file_path)
            
    for root, dirs, files in os.walk('/home/jzhe0882/Radiomics/' + dataset + '/CT'):
        for name in files:
            file_path = os.path.join(root, name)
            ct_radiomics_files.append(file_path)
            
    for root, dirs, files in os.walk('/home/jzhe0882/Radiomics/' + dataset + '/PET'):
        for name in files:
            file_path = os.path.join(root, name)
            pet_radiomics_files.append(file_path)

    pet_files = sorted(pet_files)
    ct_files = sorted(ct_files)
    centre_files = sorted(centre_files)
    centres = [np.load(c) for c in centre_files] #can load all of these into memory (other volumes are too large)
    size_files = sorted(size_files)
    bounding_box_sizes = [np.load(s) for s in size_files] #can also load all of these into memory
    mask_files = sorted(mask_files)
    pet_radiomics_files= sorted(pet_radiomics_files)
    ct_radiomics_files = sorted(ct_radiomics_files)

    #Inputs are PET/CT data, outputs are centres, sizes, masks
    X_train, X_test, y_train, y_test = train_test_split(list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files)), 
                                                        list(zip(centres, bounding_box_sizes, mask_files)), 
                                                        test_size=0.33, shuffle=True, random_state=9)
    
    pet_data['train'][dataset], ct_data['train'][dataset], pet_radiomics_data['train'][dataset], ct_radiomics_data['train'][dataset] = zip(*X_train)
    pet_data['test'][dataset], ct_data['test'][dataset], pet_radiomics_data['test'][dataset], ct_radiomics_data['test'][dataset] = zip(*X_test)
    centre_data['train'][dataset], size_data['train'][dataset], mask_data['train'][dataset] = zip(*y_train)
    centre_data['test'][dataset], size_data['test'][dataset], mask_data['test'][dataset] = zip(*y_test)

    print(pet_data['train'][dataset][0], ct_data['train'][dataset][0], pet_radiomics_data['train'][dataset][0], ct_radiomics_data['train'][dataset][0],
          centre_data['train'][dataset][0], size_data['train'][dataset][0], mask_data['train'][dataset][0])
    
    print(pet_data['test'][dataset][0], ct_data['test'][dataset][0], pet_radiomics_data['test'][dataset][0], ct_radiomics_data['test'][dataset][0],
          centre_data['test'][dataset][0], size_data['test'][dataset][0], mask_data['test'][dataset][0])

/home/jzhe0882/numpydata/HeadNeckCancer/PET/HN-CHUS-048.npy /home/jzhe0882/numpydata/HeadNeckCancer/CT/HN-CHUS-048.npy /home/jzhe0882/Radiomics/HeadNeckCancer/PET/HN-CHUS-048.npy /home/jzhe0882/Radiomics/HeadNeckCancer/CT/HN-CHUS-048.npy [ 64  49 105] [9 8 7] /home/jzhe0882/numpydata/HeadNeckCancer/Mask/HN-CHUS-048.npy
/home/jzhe0882/numpydata/HeadNeckCancer/PET/HN-HGJ-035.npy /home/jzhe0882/numpydata/HeadNeckCancer/CT/HN-HGJ-035.npy /home/jzhe0882/Radiomics/HeadNeckCancer/PET/HN-HGJ-035.npy /home/jzhe0882/Radiomics/HeadNeckCancer/CT/HN-HGJ-035.npy [70 38 61] [16 14 33] /home/jzhe0882/numpydata/HeadNeckCancer/Mask/HN-HGJ-035.npy
/home/jzhe0882/numpydata/BreastCancer/PET/10368550.npy /home/jzhe0882/numpydata/BreastCancer/CT/10368550.npy /home/jzhe0882/Radiomics/BreastCancer/PET/10368550.npy /home/jzhe0882/Radiomics/BreastCancer/CT/10368550.npy [46 58 62] [3 3 4] /home/jzhe0882/numpydata/BreastCancer/Mask/10368550.npy
/home/jzhe0882/numpydata/BreastCancer/PET/10359092.npy /home/jzhe0882/

In [6]:

def save_model_function(filepath, model):
    model.save_weights(filepath + '.h5')

    model_json = model.to_json()
    with open(filepath + '.json', 'w') as json_file:
        json_file.write(model_json)
        
def load_model_function(filepath):
    # load json and create model
    with open(filepath + '.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        
    loaded_model = model_from_json(loaded_model_json, custom_objects=custom_objects)

    # load weights into new model
    loaded_model.load_weights(filepath + '.h5')
    return loaded_model
    

In [7]:
# (x,y,z) + (x,y,z) => (x,y,z, 2)
def zip_np_volumes(vol_a, vol_b):
    vol_a = np.expand_dims(np.array(vol_a), axis=-1)
    vol_b = np.expand_dims(np.array(vol_b), axis=-1)
    return np.concatenate((vol_a, vol_b), axis=-1)

def normalise_volume(vol):
    std = np.std(vol)
    if np.isclose(std, 0):
        std = 1
    vol = np.divide(vol - np.mean(vol), std)
    return vol

volume_size = np.array([128,128,128], dtype=int)
def shift_centre(centre, shift_weight, seed=None):
    np.random.seed(seed) 
    
    centre = np.random.randn(3) * shift_weight * np.array([19.69, 4.24, 13.83]) + centre #std taken from NumpyAnalysis
    centre = np.maximum(centre, [0,0,0]) #keep centre within image bounds
    centre = np.minimum(centre, volume_size)
    return centre.astype(int)

def get_volume_input(modalities, ct_file, pet_file, ct_radiomics_file, pet_radiomics_file, volume_shape, use_radiomics):
    ct_pet = np.empty(volume_shape + (0,))
                
    if 'ct' in modalities:
        ct = np.expand_dims(np.load(ct_file), axis=-1)
        ct_pet = np.append(ct_pet, ct, axis=-1)
    if 'pet' in modalities:
        pet = np.expand_dims(np.load(pet_file), axis=-1)
        ct_pet = np.append(ct_pet, pet, axis=-1)

    if use_radiomics is not None:
        if 'ct' in modalities:
            ct_radiomics = np.expand_dims(np.load(ct_radiomics_file)[:,:,:,use_radiomics], axis=-1)
            ct_pet = np.concatenate((ct_pet, ct_radiomics), axis=-1)

        if 'pet' in modalities:
            pet_radiomics_files = np.expand_dims(np.load(pet_radiomics_file)[:,:,:,use_radiomics], axis=-1)
            ct_pet = np.concatenate((ct_pet, pet_radiomics_files), axis=-1)

    return ct_pet


In [8]:

#source_volume is a 3-dim (w,h,d) or 4-dim array (w,h,d,channels)
#generates a bounding box volume given the parameters
#centre is assumed to be relative to the source volume and rounded down
#extents is size of the bounding_box / 2
def get_bounding_box(source_volume, centre, extents):
    extents_ceil = np.ceil(extents).astype(int)
    extents_floor = extents.astype(int)
    centre = np.rint(centre).astype(int)
    
    maxima = centre + extents_ceil
    minima = centre - extents_floor
    
    #keep bounding box dimensions within the mask dimensions
    maxima = np.minimum(maxima, np.array(source_volume.shape)[:3]).astype(int)
    minima = np.maximum(minima, [0,0,0]).astype(int) 
        
    bounding_box_values = source_volume[minima[0]:maxima[0],
                                      minima[1]:maxima[1],
                                      minima[2]:maxima[2]]
    
    relative_centre = extents_floor
    relative_maxima = relative_centre + maxima - centre
    relative_minima = relative_centre + minima - centre
    
    bbox_shape = tuple(extents_ceil + extents_floor)
    if len(source_volume.shape) == 4:
        bbox_shape += (source_volume.shape[3],)
    bounding_box = np.zeros(bbox_shape)
    
    bounding_box[relative_minima[0]:relative_maxima[0],
                relative_minima[1]:relative_maxima[1],
                relative_minima[2]:relative_maxima[2]] = bounding_box_values
    
    return bounding_box

#translates the bounding box so that its values have a new reference centre
def align_bounding_box(bounding_box, box_centre, target_centre):
    displacement = (target_centre - box_centre).astype(int)
    new_box = np.copy(bounding_box)
    
    #boxes to be shifted forward have a trails of zeroes at the end of the array
    #boxes to be shifted backward have a trails of zeroes at the beginning of the array
    if displacement[0] < 0:
        new_box[displacement[0]:, :, :] = 0
    else:
        new_box[:displacement[0], :, :] = 0
                      
    if displacement[1] < 0:
        new_box[:, displacement[1]:, :] = 0
    else:
        new_box[:, :displacement[1], :] = 0
    
    if displacement[2] < 0:
        new_box[:, :, displacement[2]:] = 0
    else:
        new_box[:, :, :displacement[2]] = 0
    
    new_box = np.roll(new_box, -displacement, axis=(0,1,2))
    return new_box

#resample bounding_box to target_size while keeping aspect ratios
def resize_bounding_box(bounding_box, target_size, mode):
    size_ratio = np.amin(np.divide(target_size, bounding_box.shape))
    size_ratio_index = np.argmin(np.divide(target_size, bounding_box.shape))
    
    #the resampling here is slightly inaccurate since new_size is rounded to the nearest int
    bounding_box = sitk.GetImageFromArray(np.transpose(bounding_box))
    new_size = np.rint(size_ratio * np.array(bounding_box.GetSize()))
    new_spacing = tuple(np.multiply(np.divide(bounding_box.GetSpacing(), size_ratio), np.divide(new_size, new_size+1)))
    #additionally multiply spacing by size/(size+1) to fill out an extra row of values
        
    assert new_size[size_ratio_index] == target_size[size_ratio_index]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(bounding_box)
    resampler.SetSize((int(new_size[0]), int(new_size[1]), int(new_size[2])))
    resampler.SetOutputSpacing(new_spacing)

    if mode == 'mask':
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
        
    new_box = np.transpose(sitk.GetArrayFromImage(resampler.Execute(bounding_box)))
    
    #pads the surroundings with 0's
    return get_bounding_box(new_box, 0.5 * new_size, 0.5 * target_size)
            

## Detection Model

In [9]:

#generator for (ct,pet)->centre prediction models
def detection_generator(mode='train', dataset='HeadNeckCancer', shuffle=True, normalise=True, use_radiomics=True, 
                        modalities=['ct','pet'], batch_size=4):
    
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres):
        num_data = len(pet_files)
        volume_shape = np.load(ct_files[0]).shape
        
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres = zip(*z)

            ct_pet_batch = []
            centre_batch = []
            
            for i in range(num_data):
                ct_pet = get_volume_input(modalities, ct_files[i], pet_files[i], ct_radiomics_files[i], pet_radiomics_files[i], 
                                          volume_shape, use_radiomics)
                    
                if normalise:                    
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                centre_batch.append(centres[i])

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(centre_batch)

                    ct_pet_batch.clear()
                    centre_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(centre_batch)
    
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  centre_data[mode][dataset])

# generator for (ct,pet,centre)->(centre,size) prediction models
def localisation_generator(mode='train', dataset='HeadNeckCancer', shuffle=True, normalise=True, use_radiomics=None, batch_size=4, 
                           shift_weight=0.7, pred_centres=None, region_radius=[16,16,16], modalities=['ct', 'pet']):
    
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, shuffle, normalise, use_radiomics, batch_size):
        num_data = len(pet_files)
        volume_shape = np.load(ct_files[0]).shape
        val_seeds = list(range(num_data))
        nonlocal pred_centres
        if pred_centres is None:
            pred_centres = [None] * num_data

        while True: #loops once per epoch
            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, pred_centres, val_seeds))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, pred_centres, val_seeds = zip(*z)

            ct_pet_batch = []
            shift_centre_batch = []
            true_centre_size_batch = []
            
            for i in range(num_data):
                ct_pet = get_volume_input(modalities, ct_files[i], pet_files[i], ct_radiomics_files[i], pet_radiomics_files[i], 
                                          volume_shape, use_radiomics)
                    
                if normalise:                    
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                if pred_centres[i] is None: # generate random centres
                    if mode == 'test': #make the validation generator generate predictable centres
                        seed = val_seeds[i]
                    elif mode == 'train':
                        seed = None
                    shifted_centre = shift_centre(centres[i], shift_weight, seed)
                else:
                    shifted_centre = pred_centres[i] #use the user-defined centre
                    
                ct_pet = get_bounding_box(ct_pet, shifted_centre, np.array(region_radius))
                                
                ct_pet_batch.append(ct_pet)
                shift_centre_batch.append(shifted_centre)
                true_centre_size_batch.append(centres[i])

                if len(ct_pet_batch) == batch_size:
                    yield [np.array(ct_pet_batch), np.array(shift_centre_batch)], np.array(true_centre_size_batch)

                    ct_pet_batch.clear()
                    shift_centre_batch.clear()
                    true_centre_size_batch.clear()

            if len(ct_pet_batch) > 0:
                yield [np.array(ct_pet_batch), np.array(shift_centre_batch)], np.array(true_centre_size_batch)
    
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  centre_data[mode][dataset], size_data[mode][dataset], shuffle, normalise, use_radiomics, batch_size)

In [26]:
#test to see if generator works

batch_size = 10
dataset = 'HeadNeckCancer'
mode = 'test'
test_generator = localisation_generator(mode=mode, dataset=dataset, batch_size=batch_size, shuffle=True, use_radiomics=1, modalities=['pet'])
print('{} batches of {} samples taken over {} total samples'.format(
    math.ceil(len(ct_data[mode][dataset])/batch_size), batch_size, len(ct_data[mode][dataset])))

for i in range(2):#math.ceil(len(ct_data[mode][dataset])/batch_size)):
    ctpet, centresize = next(test_generator)
    print(i, ctpet[0].shape, ctpet[1].shape, centresize.shape)
    
test_generator = detection_generator(mode=mode, dataset=dataset, batch_size=batch_size, shuffle=True, use_radiomics=1, modalities=['ct'])
for i in range(2):#math.ceil(len(ct_data[mode][dataset])/batch_size)):
    ctpet, centre = next(test_generator)
    print(i, ctpet.shape, centre.shape)


10 batches of 10 samples taken over 98 total samples
0 (10, 32, 32, 32, 2) (10, 3) (10, 3)
1 (10, 32, 32, 32, 2) (10, 3) (10, 3)
0 (10, 128, 128, 128, 2) (10, 3)
1 (10, 128, 128, 128, 2) (10, 3)


In [10]:
# Evaluate detection model

def evaluate_detection_model(model, dataset, function='detection', batch_size=4, use_radiomics=None, modalities=['ct', 'pet']):
    mode = 'test'
    
    if function == 'detection':
        test_generator = detection_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size, 
                                             use_radiomics=use_radiomics, modalities=modalities)    
        detected_centres = model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))
        
        return str(mean_squared_error(detected_centres, centre_data[mode][dataset], multioutput='raw_values'))
    
    elif function == 'localisation':
        
        #validation_generator = localisation_generator(mode='test', dataset=dataset, batch_size=batch_size, shuffle=False, normalise=True, 
        #                                       use_radiomics=use_radiomics)
        #print(model.evaluate_generator(validation_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size)))
        
        test_generator = localisation_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size,
                                               use_radiomics=use_radiomics, modalities=modalities)    
        detected_centres = model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))

        centre_mse =  mean_squared_error(detected_centres, centre_data[mode][dataset], multioutput='raw_values')
        
        return str(centre_mse)
    
#print(evaluate_detection_model(localisation_model, 'BreastCancer', function='localisation'))


In [11]:
reset_keras()

87


In [ ]:


#cycles_per_epoch = how many times the entire training set should be cycled over for each epoch
#total_cycles = how many times the entire training set should be cycled in total

def train_detection_model(dataset, save_model=True, batch_size=4, total_epochs=100, normalise=True, shuffle=True, use_shortcut=True,
                          use_radiomics=None, transferring_model=None, loss=losses.mean_squared_error, function='detection', modalities=['ct', 'pet']):

    def get_model_name():
        return '{} {} batch={} cycles={} modalities=[] use_radiomics={} transferred={} shortcut={} {}'.format(
            function, dataset, batch_size, total_epochs, modalities, use_radiomics, transferring_model!=None, use_shortcut, loss.__name__)
        
    #define input shapes
    num_channels = len(modalities)
    if use_radiomics is not None:
        num_channels *= 2
        
    if function == 'detection':
        detection_input_shape = (128, 128, 128, num_channels)
        generator_funct = detection_generator
            
    elif function == 'localisation':
        detection_input_shape = [(32, 32, 32, num_channels), (3,)]
        generator_funct = localisation_generator

    train_generator = generator_funct(mode='train', dataset=dataset, batch_size=batch_size, shuffle=shuffle, normalise=normalise,
                                          use_radiomics=use_radiomics, modalities=modalities)
    validation_generator = generator_funct(mode='test', dataset=dataset, batch_size=batch_size, shuffle=False, normalise=normalise, 
                                               use_radiomics=use_radiomics, modalities=modalities)

    #determine whether to use old model or build new one from scratch
    if transferring_model is not None:
        detection_model = transferring_model
        #for source_layer, target_layer in zip(transferring_model.layers, detection_model.layers):
         #   target_layer.set_weights(source_layer.get_weights())
    else:
        if function == 'localisation':
            resnet_function = Resnet3DBuilder.build_resnet_34
            
        elif function == 'detection':
            resnet_function = Resnet3DBuilder.build_resnet_18

        detection_model = resnet_function(detection_input_shape, [3,3], mode=function, reg_factor=1e-8, use_shortcut=use_shortcut)
        
    detection_model.compile(optimizer='adam',
                      loss=loss)

    start_time = time.time()
    get_best = utils.GetBest(monitor='val_loss', verbose=0, mode='min')

    history = detection_model.fit_generator(train_generator, validation_data=validation_generator, verbose=1,
                                  validation_steps=math.ceil(len(ct_data['test'][dataset])/batch_size),
                                  steps_per_epoch=math.ceil(len(ct_data['train'][dataset])/batch_size), 
                                  epochs=total_epochs,
                                callbacks=[get_best])
    
    total_time = time.time() - start_time
    loss_hist = history.history['loss']
    val_loss_hist = history.history['val_loss']
    best_loss = np.amin(val_loss_hist)
    best_mse = evaluate_detection_model(detection_model, dataset, function=function, use_radiomics=use_radiomics, modalities=modalities)
    model_name = get_model_name()

    with np.printoptions(precision=2, suppress=True):
        print('loss:', np.array(loss_hist))
        print('val_loss:', np.array(val_loss_hist))
        print('best_loss:', best_loss)
        print('best_mse:', best_mse)
        
    with open('Models/keras models/training_log.txt', 'a+') as log:
        log.write('{} \ntime {}\nbest loss/mse: {} {}\ntraining: {}\nvalidation: {}\n\n'.format(
            model_name, total_time, best_loss, best_mse, loss_hist, val_loss_hist))
    
    if save_model:
        save_model_function('Models/keras models/{}'.format(model_name), detection_model)
        
    return detection_model
    

for i in range(4,7):
    blyat_model = train_detection_model(dataset='BreastCancer', use_radiomics=i, total_epochs=200, save_model=False, function='localisation',
                                       loss=losses.mean_squared_error)
    reset_keras()
    print('finished', i)

    
'''detection_model = train_detection_model(dataset='HeadNeckCancer', use_radiomics=None, total_epochs=100, save_model=True, function='detection',
                                   loss=losses.mean_squared_error)

transferring_model=detection_model
detection_model = train_detection_model(dataset='BreastCancer', use_radiomics=None, total_epochs=100, save_model=True, function='detection',
                                   loss=losses.mean_squared_error, transferring_model=transferring_model)
reset_keras()
'''


In [171]:
#print(blyat_model.get_weights()[0][0,0,0,0,0:5])

print(detection_model.get_weights()[0][0,0,0,0,0:5])
'''for layer in detection_model.get_layer('lambda_5'):
    print(layer.name)'''

[ 0.06149809  0.03975723  0.0166484  -0.15415365 -0.06149759]


"for layer in detection_model.get_layer('lambda_5'):\n    print(layer.name)"

## Segmentation Model

In [15]:
#values derived from NumpyAnalysis.ipynb
#get_best_bounding_box()
maximal_bounding_volume = np.array([32,32,32])

#generator for (ct bounding box, pet bounding_box)-> mask bounding box prediction models
def mask_bounding_box_predictor_generator(mode='train', dataset='HeadNeckCancer', shift_weight=0.7, shuffle=False, normalise=True, 
                                          use_radiomics=None, batch_size=4, region_radius=[16,16,16], pred_centres=None, modalities=['ct', 'pet']):

            
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, mask_files, centres, sizes):
        num_data = len(pet_files)
        val_seeds = list(range(num_data))
        volume_shape = np.load(ct_files[0]).shape
        
        nonlocal pred_centres
        if pred_centres is None:
            pred_centres = [None] * num_data
        
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files, pred_centres, val_seeds))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files, pred_centres, val_seeds = zip(*z)
                                
            ct_pet_batch = []
            mask_batch = []
            
            for i in range(num_data):
                
                if mode == 'test': #make the validation generator generate predictable centres
                    seed = val_seeds[i]
                elif mode == 'train':
                    seed = None
                
                #randomly sample centres/sizes around the true centre/sizes to generate bounding boxes
                if pred_centres[i] is None:
                    centre = shift_centre(centres[i], shift_weight, seed)
                else:
                    centre = pred_centres[i]
                    
                ct_pet = get_volume_input(modalities, ct_files[i], pet_files[i], ct_radiomics_files[i], pet_radiomics_files[i], 
                                          volume_shape, use_radiomics)
                ct_pet = get_bounding_box(ct_pet, centre, np.array(region_radius))
                mask = get_bounding_box(np.load(mask_files[i]), centre, np.array(region_radius))


                if normalise:
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                mask_batch.append(mask)
                
                #print(centre - centres[i])

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(mask_batch)
                    ct_pet_batch.clear()
                    mask_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(mask_batch)
                
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  mask_data[mode][dataset], centre_data[mode][dataset], size_data[mode][dataset])

In [26]:
#check to see if generator works
batch_size = 10
mode = 'train'
dataset = 'BreastCancer'
test_generator = mask_bounding_box_predictor_generator(mode, dataset, shuffle=True, normalise=False, shift_weight=0, 
                                                       batch_size=batch_size, use_radiomics=None, modalities=['pet'])
num_bg = 0.0
num_mask = 0.0

for i in range(math.ceil(len(ct_data[mode][dataset]) / batch_size)):
    ctpet, mask = next(test_generator)
               
    unique, counts = np.unique(mask, return_counts=True)
    counts = dict(zip(unique, counts))
    
    num_bg += counts[0]
    num_mask += counts.get(1,0)
    
    print(ctpet.shape, num_mask)

                      
print('mask proportion:', num_mask / (num_bg + num_mask))

(10, 32, 32, 32, 1) 2158.0
(10, 32, 32, 32, 1) 3875.0
(10, 32, 32, 32, 1) 5284.0
(10, 32, 32, 32, 1) 6794.0
(10, 32, 32, 32, 1) 7945.0
(10, 32, 32, 32, 1) 8935.0
(10, 32, 32, 32, 1) 9737.0
(10, 32, 32, 32, 1) 11188.0
(10, 32, 32, 32, 1) 13161.0
(10, 32, 32, 32, 1) 14569.0
(7, 32, 32, 32, 1) 15578.0
mask proportion: 0.004443017121787383


In [12]:
# calculates output shape given input parameters

def deconv_calculator(input_size, kernels, stride):
    input_size = np.array(input_size)
    stride = np.array(stride)

    for kernel in kernels:
        
        kernel = np.array(kernel)
        input_size = np.ceil((input_size - 1) * stride + kernel)
        print(input_size)

deconv_calculator([3,3,3], [[3,3,5],
                            [3,3,5],
                            [5,5,5],
                            [5,5,5],
                           [4,5,5]], [1,1,1])


[5. 5. 7.]
[ 7.  7. 11.]
[11. 11. 15.]
[15. 15. 19.]
[18. 19. 23.]


In [27]:

#computes the weights where if they are applied to each item, taking the mean would yield 1
#used for segmentation mask weighting
def get_class_weight(mask_proportion=0.33, precision=1000):
    
    arr = np.empty(precision)
    index = int(mask_proportion * precision)
    arr[:index] = 1
    arr[index:] = 0
    weights = class_weight.compute_class_weight('balanced', [0,1], arr)
    return weights

class_weights = get_class_weight(0.004443017121787383) #proportion taken from above
print(class_weights)

[  0.50200803 125.        ]


In [28]:


#Accepts tensors; from https://github.com/keras-team/keras/issues/3611
#Higher is better
def dice_metric(y_true, y_pred, smooth=1):
    y_pred = K.round(y_pred)

    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

#From https://gist.github.com/wassname/7793e2058c5c9dacb5212c0ac0b18a8a
def dice_coef(y_true, y_pred, smooth=100):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    y_true = K.batch_flatten(y_true)
    y_pred = K.batch_flatten(y_pred)
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return K.mean((2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


#From https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    y_true = K.batch_flatten(y_true)
    y_pred = K.batch_flatten(y_pred)
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = K.mean((intersection + smooth) / (sum_ - intersection + smooth))
    return (1 - jac) * smooth



#from https://stackoverflow.com/questions/46009619/keras-weighted-binary-crossentropy
def weighted_binary_crossentropy(y_true, y_pred):
    zero_weight = class_weights[0]
    one_weight = class_weights[1]
    
    # Calculate the binary crossentropy
    b_ce = K.binary_crossentropy(y_true, y_pred)
    
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce

    # Return the mean error
    return K.mean(weighted_b_ce)

def blyat():

    # Test
    # Test
    print("TYPE                 |Almost_right |half right |all_wrong")
    y_true = np.array([[1,0,0,0,0,0,0,0,0,0,0]])
    y_pred = np.array([[1,0,0,0,0,0,0,0,0,0,0]])
    
    r = (
        K.variable(y_true) *
        K.variable(y_pred)
    ).eval(session=K.get_session())
    print('mult',r)
    
    r = jaccard_distance_loss(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('jaccard_distance_loss',r)
    #assert r[0]<r[1]
    #assert r[1]<r[2]

    r = keras.losses.binary_crossentropy(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('binary_crossentropy',r)
    print('binary_crossentropy_scaled',r/r.max())
    #assert r[0]<r[1]
    #assert r[1]<r[2]
    
    r = weighted_binary_crossentropy(
        K.variable(y_true),
        K.variable(y_pred),
    ).eval(session=K.get_session())
    print('weighted_binary_crossentropy',r)
    print('weighted_binary_crossentropy_scaled',r/r.max())
    
    ups = np.arange(8).reshape((1,2,2,2))
    r = resnet3d4.upsample3d(2)(
        K.variable(ups)
    ).eval(session=K.get_session())
    print('upsampled',r)
    
blyat()

TYPE                 |Almost_right |half right |all_wrong
mult [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
jaccard_distance_loss 0.0
binary_crossentropy [1.0174631e-07]
binary_crossentropy_scaled [1.]
weighted_binary_crossentropy 1.4002886e-06
weighted_binary_crossentropy_scaled 1.0
upsampled [[[[0. 0. 1. 1.]
   [0. 0. 1. 1.]
   [2. 2. 3. 3.]
   [2. 2. 3. 3.]]

  [[0. 0. 1. 1.]
   [0. 0. 1. 1.]
   [2. 2. 3. 3.]
   [2. 2. 3. 3.]]

  [[4. 4. 5. 5.]
   [4. 4. 5. 5.]
   [6. 6. 7. 7.]
   [6. 6. 7. 7.]]

  [[4. 4. 5. 5.]
   [4. 4. 5. 5.]
   [6. 6. 7. 7.]
   [6. 6. 7. 7.]]]]


In [29]:
#Evaluate segmentation model

#Accepts numpy arrays
#Higher is better
def dice_eval(true_mask, pred_mask, pred_centres):
    pred_mask = np.rint(pred_mask)
    true_mask = np.rint(true_mask)
    
    true_values = np.unique(true_mask)
    pred_values = np.unique(pred_mask)
    
    #when training, if both true and predicted masks are 0's this is the best prediction
    #when evaluating, this indicates that the predicted centre is outside of the scope of the mask; worst prediction
    if np.array_equal(true_values, [0]) and np.array_equal(pred_values, [0]):
        if pred_centres is None:
            return 1 
        else:
            return 0 
    
    pred_mask = pred_mask.flatten()
    true_mask = true_mask.flatten()
    ret = 1 - distance.dice(pred_mask, true_mask)
    return ret

#returns dice acc, dice std, and best image segmentation
def evaluate_segmentation_model(dataset, model, mode='test', output_segmentation=False, fill_holes=False, use_radiomics=None,
                                pred_centres=None, shift_weight=0.7, region_radius=[16,16,16], modalities=['ct', 'pet']):
    
    validation_generator = mask_bounding_box_predictor_generator(mode, dataset, shuffle=False, 
                                                                 use_radiomics=use_radiomics, normalise=True, batch_size=1,
                                                                 pred_centres=pred_centres, shift_weight=shift_weight, modalities=modalities)
    dices = []
    max_acc = 0
    max_patient = ''
        
    for i in range(len(ct_data[mode][dataset])):
        ctpet, true_mask = next(validation_generator)
        pred_mask = model.predict(ctpet)[0]
        
        if pred_centres is not None: #shift the true bounding box so that it is centres with pred_centres
            true_mask = get_bounding_box(np.load(mask_data[mode][dataset][i]), pred_centres[i], np.array(region_radius))
        else:
            true_mask = true_mask[0]
        
        if fill_holes:
            pred_mask = binary_fill_holes(pred_mask)
        
        dice_index = dice_eval(true_mask, pred_mask, pred_centres)
        dices.append(dice_index)
                
        if dice_index > max_acc:
            max_acc = dice_index
            max_patient = ct_data[mode][dataset][i]
        
        if output_segmentation:
            patient_name = os.path.splitext(os.path.basename(pet_data[mode][dataset][i]))[0]
            mask_radius = 0.5 * np.array((32, 32, 32))
            utils.output_numpy_mask_to_nrrd(patient_name, np.rint(pred_mask), centre_data[mode][dataset][i], mask_radius, dataset)
            utils.output_numpy_mask_to_nrrd(patient_name, true_mask, centre_data[mode][dataset][i], mask_radius, dataset, filename_tag='-true')
            
            
    return np.average(dices), np.std(dices), os.path.basename(max_patient), max_acc

#print('dice accuracy, dice slice-by-slice accuracy, most accurate:')
#print(evaluate_segmentation_model('BreastCancer', segmentation_model, 'test', output_segmentation=False))




In [34]:
reset_keras()

878


In [12]:
#used for reloading models
custom_objects={'jaccard_distance_loss' : jaccard_distance_loss,
                                    'dice_coef_loss': dice_coef_loss,
                                          'dice_metric': dice_metric,
                'weighted_binary_crossentropy' : weighted_binary_crossentropy,
                                          'dice_coef': dice_coef,
                                       'conv4d' : conv4d.conv4d,
                                    'l2': l2,
                                    'Resnet3DBuilder': Resnet3DBuilder,
                                    'tf' : tf,
                                    'upsample3d_helper' : resnet3d4.upsample3d_helper}



In [132]:
mode = 'test'
dataset = 'BreastCancer'
batch_size = 4

In [13]:
#CANT LOAD THIS SHIT CONV4D DOES NOT LOAD WEIGHTS PROPERLY BLYAAAAAT
#detection_model = load_model_function('/home/jzhe0882/Models/keras models/detection BreastCancer batch=4 cycles=1 shuffle=True normalise=True use_radiomics=None transferred=False mean_squared_error')

#detection model test
test_generator = detection_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size, use_radiomics=None)    
detected_centres = detection_model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))   
centre_mse =  mean_squared_error(detected_centres, centre_data[mode][dataset], multioutput='raw_values')

print(centre_mse)

[36.42391753  6.94835554 20.97476426]


In [ ]:
#save detected centres

test_generator = detection_generator(mode=mode, dataset=dataset, shuffle=False, normalise=True, batch_size=batch_size, use_radiomics=None)    
detected_centres = detection_model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))   
centre_mse =  mean_squared_error(detected_centres, centre_data[mode][dataset], multioutput='raw_values')


print(centre_mse)

In [24]:

#np.save('Models/keras models/detected_centres.npy', detected_centres)

detected_centres = np.load('Models/keras models/detected_centres.npy')

centre_mse =  mean_squared_error(blyat_centres, centre_data[mode][dataset], multioutput='raw_values')
print(centre_mse)

[36.42391753  6.94835554 20.97476426]


In [ ]:
#(unsuccessfully) successively refine predicted centres

loops=100

def successive_refinement(detected_centres, loops=100):
    successive_refined_centres = np.empty((loops, len(centre_data[mode][dataset]), 3))
    current_prediction = detected_centres
    
    for i in range(loops):
        test_generator = localisation_generator(mode=mode, dataset=dataset, batch_size=batch_size, 
                                                shuffle=False, normalise=True, use_radiomics=None, pred_centres=current_prediction)
        current_prediction = detection_model.predict_generator(test_generator, steps=math.ceil(len(ct_data[mode][dataset])/batch_size))
        successive_refined_centres[i] = current_prediction
        
        print(mean_squared_error(current_prediction, centre_data[mode][dataset], multioutput='raw_values'))
    return successive_refined_centres

refined_centres = successive_refinement(detected_centres)
np.save('Models/keras models/refined_centres.npy', refined_centres)


In [40]:
#save refinement losses

refined_centres = np.load('Models/keras models/refined_centres.npy')
refined_centres_mses = np.empty((loops, 3))
refined_centres_diffs = np.empty((loops, len(centre_data[mode][dataset]), 3))

for i in range(len(refined_centres)):
    centre_mse =  mean_squared_error(refined_centres[i], centre_data[mode][dataset], multioutput='raw_values')
    refined_centres_mses[i] = centre_mse
    refined_centres_diffs[i] = centre_data[mode][dataset] - refined_centres[i]
    
np.save('Models/keras models/refined_centres_mses.npy', refined_centres_mses)
np.save('Models/keras models/refined_centres_diffs.npy', refined_centres_diffs)

In [ ]:
print(evaluate_segmentation_model(dataset, segmentation_model, mode, output_segmentation=False, fill_holes=False, 
                                                  use_radiomics=None, pred_centres=refined_centres[0]))

In [ ]:
#check how shift_weight varies each prediction

dice_indices = np.empty((100))
for i in range(100):
    
    blyat_centres = []
    for j in range(len(centre_data[mode][dataset])):
        centre = shift_centre(centre_data[mode][dataset][j], shift_weight=0.01*i, seed=j)
        blyat_centres.append(centre)
        
    dice_indices[i],_,_,_ = evaluate_segmentation_model(dataset, segmentation_model, mode=mode, output_segmentation=False, fill_holes=False, 
                                                  use_radiomics=None, pred_centres=blyat_centres)
    print(dice_indices[i])
    
np.save('Models/keras models/dice_indices.npy', dice_indices)

# Playground

In [ ]:
print(detection_model.get_layer(index=-4))

In [ ]:
#old validation generator for resizing stuff
maximal_bounding_volume = np.array([32,32,32])

#generator for (ct bounding box, pet bounding_box)-> mask bounding box prediction models
def mask_bounding_box_predictor_generator(mode='train', dataset='HeadNeckCancer', shift_centres=True, shift_sizes=True, shift_weight=0.7, 
                                          shuffle=False, normalise=True, use_radiomics=True, batch_size=4, region_radius=[16,16,16]):

    def shift_centre(centre):
        centre = np.random.randn(3) * shift_weight * np.array([20.4,4.6,21.2]) + centre #std taken from NumpyAnalysis
        centre = np.maximum(centre, [0,0,0]) #keep centre within image bounds
        centre = np.minimum(centre, volume_size)
        return centre.astype(int)
    
    def shift_size(size):
        size = np.random.randn(3) * shift_weight *  np.array([3.6,3,5.7]) + size #std taken from NumpyAnalysis
        size = np.maximum(size, [1,1,1]) #keep size to > 1
        size = np.minimum(size, maximal_bounding_volume)
        return size
        
    def get_resized_bounding_box(source_volume, centre, size, target_size, mode):
        bounding_box = get_bounding_box(source_volume, centre, 0.5 * size)
        return resize_bounding_box(bounding_box, target_size, mode)
            
    def helper(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, mask_files, centres, sizes, 
               shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size):
        while True:

            if shuffle:
                z = list(zip(pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files))
                random.shuffle(z)
                pet_files, ct_files, pet_radiomics_files, ct_radiomics_files, centres, sizes, mask_files = zip(*z)
                                
            ct_pet_batch = []
            mask_batch = []
            
            for i in range(len(pet_files)):
                
                #randomly sample centres/sizes around the true centre/sizes to generate bounding boxes
                if shift_centres:
                    centre = shift_centre(centres[i], shift_weight,)
                else:
                    centre = centres[i]

                if shift_sizes:
                    size = shift_size(sizes[i])
                else:
                    size = sizes[i]
  

                ct = get_resized_bounding_box(np.load(ct_files[i]), centre, size, maximal_bounding_volume, 'ct')
                pet = get_resized_bounding_box(np.load(pet_files[i]), centre, size, maximal_bounding_volume, 'pet')
                mask = get_resized_bounding_box(np.load(mask_files[i]), centre, size, maximal_bounding_volume, 'mask')
                
                ct_pet = zip_np_volumes(ct, pet)
                
                if use_radiomics:
                    ct_radiomics = np.load(ct_radiomics_files[i])
                    pet_radiomics = np.load(pet_radiomics_files[i])
                    
                    for axis in range(ct_radiomics.shape[-1]):
                        ct_radiomic_feature = get_resized_bounding_box(ct_radiomics[:,:,:,axis], centre, size, maximal_bounding_volume, 'ct')
                        pet_radiomic_feature = get_resized_bounding_box(pet_radiomics[:,:,:,axis], centre, size, maximal_bounding_volume, 'pet')
                        ct_radiomic_feature = np.expand_dims(ct_radiomic_feature, axis=-1)
                        pet_radiomic_feature = np.expand_dims(pet_radiomic_feature, axis=-1)
                        
                        ct_pet = np.concatenate((ct_pet, ct_radiomic_feature, pet_radiomic_feature), axis=-1)
                
                if normalise:
                    for axis in range(ct_pet.shape[-1]):
                        ct_pet[:,:,:,axis] = normalise_volume(ct_pet[:,:,:,axis])
                
                ct_pet_batch.append(ct_pet)
                mask_batch.append(mask)

                if len(ct_pet_batch) == batch_size:
                    yield np.array(ct_pet_batch), np.array(mask_batch)
                    ct_pet_batch.clear()
                    mask_batch.clear()

            if len(ct_pet_batch) > 0:
                yield np.array(ct_pet_batch), np.array(mask_batch)
                
    return helper(pet_data[mode][dataset], ct_data[mode][dataset], pet_radiomics_data[mode][dataset], ct_radiomics_data[mode][dataset],
                  mask_data[mode][dataset], centre_data[mode][dataset], size_data[mode][dataset],
                 shift_centres, shift_sizes, shuffle, normalise, use_radiomics, batch_size)

In [189]:
blyat = np.zeros((1,1,1))
print(blyat)
np.append(blyat, [[[2]]], axis=-1)

[[[0.]]]


array([[[0., 2.]]])